In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from astropy.table import Table
import os
import numpy as np
import cartopy.crs as ccrs
import json
import os
import seaborn as sns

In [2]:
from astropy.table import Table

In [3]:
os.chdir('/home/walml/repos/zoobot')

## Remove DR1/2 repeats

In [4]:
dr1_dr2 = Table.read('/media/walml/beta/galaxy_zoo/decals/catalogs/raw/decals_dr1_and_dr2.csv').to_pandas()

/home/walml/anaconda3/envs/zoobot/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'background' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/walml/anaconda3/envs/zoobot/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'bad_image' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/walml/anaconda3/envs/zoobot/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'band' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/walml/anaconda3/envs/zoobot/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'bands' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/walml/anaconda3/envs/zoobot/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableRe

In [5]:
del dr1_dr2['_id']

In [6]:
(dr1_dr2['iauname'].value_counts() > 1).sum()

2413

In [7]:
counts = dr1_dr2['iauname'].value_counts() > 1
repeating_iaunames = counts[counts].index.values
repeating_iaunames = [a for a in repeating_iaunames]

In [8]:
len(repeating_iaunames)

2413

In [9]:
repeating_subject_ids = dr1_dr2[dr1_dr2['iauname'].isin(repeating_iaunames)]['subject_id']
len(repeating_subject_ids), len(set(repeating_subject_ids))

(4826, 4826)

In [10]:
dr1_dr2[dr1_dr2['iauname'] == 'J154253.87+060230.5'].squeeze()

,col0,absolute_size,background,bad_image,band,bands,camera,counters,data_release,dec,...,sdss_dr7_id,sdss_dr8_id,sdss_id,sfr,simulation,snapshot,subdir,subhalo_id,survey,subject_id
26666,26666,10.090934,NaN,NaN,NaN,NaN,NaN,"{""feature"": 0, ""smooth"": 0, ""star"": 0}",DR1,6.041827,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,decals,55a7d7ae7a726119cd00682a
70144,70144,10.090934,NaN,NaN,NaN,NaN,NaN,"{""star"": 0, ""smooth"": 0, ""feature"": 0}",DR2,6.041827,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,decals,56f3dcb25925d900420269df


Different: _id, data_release, nobs_max (new), retire_at
    
This repeat is a galaxy uploaded in both DR1 and DR2. That's always true:


In [11]:
dr1_dr2.duplicated('iauname').sum(), dr1_dr2.duplicated(subset=['iauname', 'data_release']).sum()

(2413, 0)

In [12]:
dr1_dr2['data_release'].value_counts()

DR2    61595
DR1    42117
Name: data_release, dtype: int64

Let's remove the DR1 duplicates from the DR1 classifications file

In [13]:
dr1 = pd.read_csv('/media/walml/beta/galaxy_zoo/decals/gzreduction_ouroborous/working_dir/raw/classifications/2017-10-15_galaxy_zoo_decals_classifications.csv')
dr2 = pd.read_csv('/media/walml/beta/galaxy_zoo/decals/gzreduction_ouroborous/working_dir/raw/classifications/2017-10-15_galaxy_zoo_decals_dr2_classifications.csv')

In [14]:
dr1.columns.values

array(['id', 'subject_id', 'user', 'created_at', 'lang', 'user_agent',
       'decals-0', 'decals-1', 'decals-2', 'decals-3', 'decals-4',
       'decals-5', 'decals-6', 'decals-7', 'decals-8', 'decals-9',
       'decals-10', 'decals-11'], dtype=object)

In [17]:
dr1['decals-0'].value_counts()

a-0    914957
a-1    489894
a-2    312316
a-3         4
Name: decals-0, dtype: int64

In [18]:
dr2['decals-0'].value_counts()

a-0    1284174
a-1     683792
a-2     507486
a-3          8
Name: decals-0, dtype: int64

In [ ]:
# all classified subjects are in the subject catalog
dr1['subject_id'].isin(dr1_dr2['subject_id']).mean(), dr2['subject_id'].isin(dr1_dr2['subject_id']).mean()

In [ ]:
dr1_with_subjects = pd.merge(dr1, dr1_dr2, on='subject_id', how='inner')

In [ ]:
dr1_subjects_per_iauname = dr1_with_subjects.groupby('subject_id').agg({'iauname': lambda x: len(set(x))}).reset_index()
dr1_subjects_per_iauname['iauname'].value_counts()

In [ ]:
dr2_with_subjects = pd.merge(dr2, dr1_dr2, on='subject_id', how='inner')

In [ ]:
dr2_subjects_per_iauname = dr2_with_subjects.groupby('subject_id').agg({'iauname': lambda x: len(set(x))}).reset_index()
dr2_subjects_per_iauname['iauname'].value_counts()

In [ ]:
set(dr1_with_subjects['iauname']).intersection(dr2_with_subjects['iauname'])

So the subject catalog includes subject_ids with the same iauname (duplicates one), but the recorded classifications do not?

All but 1 DR2 subject has been classified, but 10k DR1 galaxies have not (42k vs 32k).

In [ ]:
dr1_subjects = dr1_dr2.query('data_release == "DR1"')
len(dr1_subjects), dr1_subjects['iauname'].isin(repeating_iaunames).sum()

In [ ]:
dr1_subjects_not_classified = dr1_subjects[~dr1_subjects['subject_id'].isin(dr1['subject_id'])]
len(dr1_subjects_not_classified), dr1_subjects_not_classified['iauname'].isin(repeating_iaunames).sum()


All the repeating subjects were not classified, so I assume they noticed beforehand and removed them! Good times. So ultimately, *there's no work needed to tweak the classifications*.

However, there are still 7.5k of 42k DR1 subjects with no classifications. Why is that?

In [ ]:
dr1_subjects_not_classified.iloc[0]

In [ ]:
_, bins = np.histogram(np.linspace(0, 10), bins=30)
dr1_subjects_not_classified['petrorad_r'].hist(bins=bins, density=True, alpha=.5)
dr1_subjects['petrorad_r'].hist(bins=bins, density=True, alpha=.5)

In [ ]:
sns.scatterplot(data=dr1_subjects_not_classified, x='ra', y='dec', alpha=.3)

In [ ]:
sns.scatterplot(data=dr1_subjects, x='ra', y='dec', alpha=.3)

There's no obvious trend in size or location. For DR2, about 1/3rd of galaxies had bad images, and this would give about the right number of missing galaxies here. Perhaps the subject catalog includes those bad-image galaxies. 

Adam has now sent the DR1 images, so we can check...except it doesn't include these pics. So they were definitely never even uploaded.

In [ ]:
dr1_subjects_not_classified['iauname']

In [ ]:
assert False

# Aggregated 

I used these files to produce aggregated classifications:

In [ ]:
# nope, this is for Nair comparison - same for panoptes probably
# aggregated = pd.read_csv('/media/walml/beta/galaxy_zoo/decals/gzreduction_ouroborous/working_dir/output/dr2_predictions_with_subject.csv')

In [23]:
# also has a version with predictions added, most likely etc
aggregated = pd.read_csv('/media/walml/beta/galaxy_zoo/decals/gzreduction_ouroborous/working_dir/votes/dr2_aggregated_votes.csv')

In [24]:
aggregated

,subject_id,smooth-or-featured_smooth,smooth-or-featured_featured-or-disk,smooth-or-featured_artifact,how-rounded_completely,how-rounded_in-between,how-rounded_cigar-shaped,disk-edge-on_yes,disk-edge-on_no,edge-on-bulge_rounded,...,spiral-arm-count_more-than-4_fraction,bulge-size_total-votes,bulge-size_none_fraction,bulge-size_obvious_fraction,bulge-size_dominant_fraction,merging_total-votes,merging_merger_fraction,merging_tidal-debris_fraction,merging_both_fraction,merging_neither_fraction
0,55a7d7a27a726119cd00000b,19,15,11,16,2,1,2,13,2,...,0.125,13,0.000000,0.923077,0.076923,34,0.147059,0.088235,0.088235,0.676471
1,55a7d7a27a726119cd00000c,33,2,6,28,5,0,1,1,1,...,0.000,1,0.000000,1.000000,0.000000,35,0.200000,0.085714,0.028571,0.685714
2,55a7d7a27a726119cd00000d,36,1,10,23,13,0,0,1,0,...,0.000,1,0.000000,1.000000,0.000000,37,0.027027,0.027027,0.000000,0.945946
3,55a7d7a27a726119cd00000e,32,15,6,16,16,0,0,15,0,...,0.250,15,0.266667,0.733333,0.000000,47,0.021277,0.042553,0.021277,0.914894
4,55a7d7a27a726119cd00000f,40,6,7,10,29,1,0,6,0,...,0.000,6,0.166667,0.666667,0.166667,46,0.086957,0.043478,0.021739,0.847826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94019,56f3deb65925d90042037015,6,12,22,4,1,1,0,12,0,...,0.000,12,0.500000,0.500000,0.000000,18,0.277778,0.111111,0.000000,0.611111
94020,56f3deb65925d90042037017,21,15,4,17,4,0,0,15,0,...,0.000,15,0.333333,0.600000,0.066667,36,0.277778,0.027778,0.083333,0.611111
94021,56f3deb65925d90042037019,22,15,3,2,16,4,11,4,11,...,0.000,4,0.250000,0.750000,0.000000,37,0.540541,0.000000,0.108108,0.351351
94022,56f3deb65925d9004203701b,30,2,8,26,3,1,0,2,0,...,0.000,2,0.000000,0.000000,1.000000,32,0.093750,0.093750,0.000000,0.812500


In [25]:
aggregated['smooth-or-featured_artifact'].sum()

819802

In [26]:
len(set(dr1['subject_id']))

32430